<center><img src="./logo.png" alt="Header" style="width: 800px;"/></center>

In [1]:
import cv2
import ipywidgets.widgets as widgets
import threading
import time
import numpy as np

image_widget = widgets.Image(format='jpeg', width=600, height=500)  
display(image_widget)                                              

Image(value=b'', format='jpeg', height='500', width='600')

In [2]:
#bgr8转jpeg格式 #bgr8 to jpeg format
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [3]:
image = cv2.VideoCapture(0)                           

width=600
height=500

ret, frame = image.read()    
image_widget.value = bgr8_to_jpeg(frame)


In [4]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Arm_Lib import Arm_Device

Arm = Arm_Device()
time.sleep(.1)

In [5]:
def arm_clamp_block(enable):
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 60, 400)
    else:
        Arm.Arm_serial_servo_write(6, 135, 400)
    time.sleep(.5)

    
def arm_move(p, s_time = 500):
    for i in range(5):
        id = i + 1
        if id == 5:
            time.sleep(.1)
            Arm.Arm_serial_servo_write(id, p[i], int(s_time*1.2))
        else :
            Arm.Arm_serial_servo_write(id, p[i], s_time)
        time.sleep(.01)
    time.sleep(s_time/1000)

def arm_move_up():
    Arm.Arm_serial_servo_write(2, 90, 1500)
    Arm.Arm_serial_servo_write(3, 90, 1500)
    Arm.Arm_serial_servo_write(4, 90, 1500)
    time.sleep(.1)


In [6]:
p_mould = [90, 120, 0, 0, 90]
arm_clamp_block(0)
arm_move(p_mould, 1000)
time.sleep(1)


In [7]:
def update_camera():
    while 1:
        ret, frame = image.read()
        
        if ret:
            # Convert the frame to HSV color space
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            
            color_hsv = {"red" : ((160, 43, 46), (180, 255, 255)),
                        "green" : ((40, 43, 46), (90, 255, 255)),
                        "blue" : ((100, 43, 46), (130, 255, 255)),
                        "yellow": ((20, 43, 46), (35, 255, 255))}            
            
           # Create masks and find contours for each color
            for color, (lower, upper) in color_hsv.items():
                mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                # Define rectangle color based on detected color
                rect_color = {
                    "red": (0, 0, 255),
                    "green": (0, 255, 0),
                    "blue": (255, 0, 0),
                    "yellow": (0, 255, 255)
                }[color]

                # Draw rectangles around detected colors
                for contour in contours:
                    if cv2.contourArea(contour) > 100:  # Adjust the size threshold as needed
                        x, y, w, h = cv2.boundingRect(contour)
                        cv2.rectangle(frame, (x, y), (x+w, y+h), rect_color, 2)

            # Calculate the center coordinates of the frame
            center_x, center_y = int(frame.shape[1] / 2), int(frame.shape[0] / 2)

            # Get the HSV value of the center pixel
            center_color = hsv[center_y, center_x]

            # Display the HSV value on the frame
            text = f"Center HSV: {center_color}"
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (255, 255, 255), 2, cv2.LINE_AA)

            image_widget.value = bgr8_to_jpeg(frame)
            time.sleep(0.010)
            
        
import threading
thread = threading.Thread(target=update_camera)
thread.start()



In [8]:
# #release
# #释放
# image.release()            
# cv2.destroyAllWindows()